In [66]:
import neurokit2 as nk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)



from tqdm.notebook import tqdm



def processECG(data): 
    
    
    ecg_signal = data['ecgArray']
    
    try:
        _, info = nk.ecg_process(ecg_signal, sampling_rate = 200)
        rpeaks = info["ECG_R_Peaks"]
        hrv_indices_time = nk.hrv_time(rpeaks, sampling_rate = 200)
        meannn = hrv_indices_time["HRV_MeanNN"][0]
        pnn50 = hrv_indices_time["HRV_pNN50"][0]
        test_HRV = meannn, pnn50
    except:
        test_HRV = 'None', 'None'
    
    sampling_rate = 200  
    method = 'neurokit'
    
    try:
        ecg_signal = nk.signal_sanitize(ecg_signal)
        ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate = sampling_rate, method = method)
    except Exception as e:
        return { "HR": 'None', 'ECG_Processed' : 'None', 'ECG_MeanNN' : test_HRV[0], 'ECG_pNN50' : test_HRV[1], 
                'Short_Message' : 'ECG_Filtering_Error'}
    
    # Detect R-peaks
    try:
        _, info = nk.ecg_peaks(
            ecg_cleaned = ecg_cleaned,
            sampling_rate = sampling_rate,
            method = method,
            correct_artifacts = True,
        )
    except Exception as e:
        return { "HR": 'None', 'ECG_Processed' : list(ecg_cleaned), 'ECG_MeanNN' : test_HRV[0], 'ECG_pNN50' : test_HRV[1], 
                'Short_Message' : 'ECG_Detecting_Peaks_Error'}
    
    # Calculate Heart Rate
    try:    
        rate = nk.signal_rate(
            info, sampling_rate=sampling_rate, desired_length=len(ecg_cleaned)
        )
        rate = rate.mean()
        
        rate = round(rate, 1)
        
    except Exception as e:
        return { "HR": 'None', 'ECG_Processed' : list(ecg_cleaned), 'ECG_MeanNN' : test_HRV[0], 'ECG_pNN50' : test_HRV[1], 
                'Short_Message' : 'ECG_Computing_HeartRate_Error'}
    
    if np.isnan(rate):
        return { "HR": 'None', 'ECG_Processed' : list(ecg_cleaned), 'ECG_MeanNN' : test_HRV[0], 'ECG_pNN50' : test_HRV[1], 
                'Short_Message' : 'ECG_Computing_HeartRate_Error'}
        
    return { "HR": rate, 'ECG_Processed' : list(ecg_cleaned), 'ECG_MeanNN' : test_HRV[0], 'ECG_pNN50' : test_HRV[1],
            'Short_Message' : 'SUCCESS'}

In [63]:
dataset = pd.read_json('20240610.json')
dataset.columns

Index(['_id', 'userId', 'createdAt', 'data', 'userCancelledAt',
       'userRemovedAt'],
      dtype='object')

In [64]:
def get_signals(row):
    try:
        data = row['data']
        ecg = data['ecg']['ecgArray']
        irppg = data['ppg']['irArray']
        redppg = data['ppg']['redArray']
        result = {
            'ecg' : ecg,
            'irppg' : irppg,
            'redppg' : redppg 
        }
    except:
        result = {
            'ecg' : None,
            'irppg' : None,
            'redppg' : None 
        }
    return pd.Series(result)

signals = dataset.apply(get_signals, axis = 1).dropna()
signals['ecg_std'] = signals.apply(lambda x : np.std(x['ecg']), axis = 1)
signals.drop(signals[signals['ecg_std'] < 10].index, inplace = True)
signals['ECG'] = signals.apply(lambda x : {'ecgArray' : x[0]}, axis = 1)
signals

,ecg,irppg,redppg,ecg_std,ECG
2,"[32767, 32767, 32767, 32767, 32767, 32767, 327...","[4454, 4456, 4458, 4453, 4455, 4454, 4450, 445...","[17263, 17249, 17255, 17274, 17241, 17233, 172...",2961.478529,"{'ecgArray': [32767, 32767, 32767, 32767, 3276..."
3,"[31675, 31498, 31278, 31641, 31688, 31013, 312...","[10453, 10456, 10456, 10472, 10469, 10491, 104...","[21951, 21945, 21906, 22091, 21890, 22010, 220...",2086.989928,"{'ecgArray': [31675, 31498, 31278, 31641, 3168..."
4,"[19599, 19582, 20035, 19607, 19251, 19648, 196...","[14590, 14593, 14608, 14619, 14637, 14657, 146...","[26419, 26494, 26433, 26489, 26598, 26422, 265...",1010.400505,"{'ecgArray': [19599, 19582, 20035, 19607, 1925..."
8,"[17195, 17801, 17948, 17230, 17609, 17976, 171...","[19959, 19801, 19614, 19448, 19344, 19246, 191...","[-28463, -28561, -28958, -29200, -29336, -2938...",1893.426719,"{'ecgArray': [17195, 17801, 17948, 17230, 1760..."
9,"[6220, 5821, 6170, 6561, 6059, 6079, 6562, 627...","[9851, 9846, 9847, 9834, 9837, 9845, 9857, 985...","[27113, 27000, 27003, 27175, 27285, 27033, 270...",1082.868275,"{'ecgArray': [6220, 5821, 6170, 6561, 6059, 60..."
...,...,...,...,...,...
1567,"[-722, -682, -756, -796, -743, -741, -789, -77...","[-31537, -31535, -31544, -31561, -31550, -3154...","[-24173, -24171, -24155, -24317, -24338, -2420...",239.000731,"{'ecgArray': [-722, -682, -756, -796, -743, -7..."
1568,"[-149, -128, -94, -134, -153, -115, -114, -154...","[12619, 12634, 12630, 12644, 12638, 12645, 126...","[20802, 20953, 21001, 21008, 21002, 21000, 209...",118.265840,"{'ecgArray': [-149, -128, -94, -134, -153, -11..."
1572,"[14105, 14570, 14973, 14599, 14480, 14888, 146...","[17984, 17966, 17967, 17976, 17965, 17967, 179...","[26419, 26541, 26555, 26560, 26577, 26604, 262...",1651.268366,"{'ecgArray': [14105, 14570, 14973, 14599, 1448..."
1577,"[6982, 7221, 7596, 7630, 7688, 7647, 7646, 769...","[16566, 16558, 16560, 16568, 16564, 16568, 165...","[-30945, -29727, -29763, -29647, -29752, -2963...",2309.877377,"{'ecgArray': [6982, 7221, 7596, 7630, 7688, 76..."


In [69]:
processECG(signals['ECG'].iloc[1])['ECG_pNN50']

22.22222222222222